# Manejo de datos depositados en una Base de datos 

## 2. Conceptos clave: BioProject, BioSample, SRA y GEO
De manera general, un BioProject (PRJNAxxxxxx) describe el proyecto de investigación en su conjunto (por ejemplo, un estudio de RNA-Seq) y se vincula a una o varias BioSamples (SAMNxxxxxx), que documentan las características biológicas de cada muestra. A partir de estas BioSamples se generan los datos crudos de secuenciación, almacenados en el Sequence Read Archive (SRA, SRRxxxxxx).

En algunos casos, el proyecto también deposita información en Gene Expression Omnibus (GEO):

Cada GSE corresponde a un experimento completo.
Cada GSM corresponde a una muestra individual, asociada a sus archivos de secuenciación.
En resumen, la ruta BioProject → BioSample → SRA está siempre presente, mientras que la rama hacia GEO es opcional y depende del tipo de estudio. 



## Entrez 
Es un modulo de **biopython**  que permite efecturar descarga de base de datos  

### Resumen de funciones de Bio.Entrez
- `esearch`: Buscar IDs en una base de datos.
- `efetch`: Descargar información completa (texto, XML, secuencias, metadatos).
- `esummary`: Obtener un resumen de un ID (título, organismo, etc.). 
> La función esummary devuelve un diccionario que resume la información esencial de un BioProject, incluyendo datos sobre el organismo, tipo y objetivo del > > proyecto, fecha de registro, estado de secuenciación y más. La siguiente estructura muestra los campos más relevantes obtenidos para el proyecto
- `elink`: Encontrar vínculos entre bases de datos (ej. GSE → SRA).
- `egquery`: Buscar un término en todas las bases del NCBI y ver cuántos resultados hay.
- `espell`: Corregir errores ortográficos en la búsqueda.
- `einfo`: Ver información de una base de datos (campos, descripción, qué contiene).
- `read`: convierte la respuesta de NCBI, en un objeto de python(listas, diccionarios, strings, etc.).

In [ ]:
from Bio import Entrez 
import pandas as pd 
from io import StringIO 
import subprocess
import os 

Entrez.email= "ismadls@lcg.unam.mx"

proyec_acc= "PRJNA552284" 

#Hacemos la consulta con esaearch de el id en la base bioproject en especifico 
handle= Entrez.esearch(db="bioproject", term=proyec_acc)
#guardamos 
search= Entrez.read(handle) 
handle.close()

print(search)

#salida de la consulta 
"""
'Count': '1', 
'RetMax': '1', 
'RetStart': '0', 
'IdList': ['552284'], 
'TranslationSet': [], 
'TranslationStack': [{'Term': 'PRJNA552284[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'PRJNA552284[All Fields]'}

""" 

#consulatmos el UID 
project_uid= search["IdList"][0] 

print(project_uid)

#consultamos el resumen del proyecto 

handle = Entrez.esummary(db="bioproject", id=project_uid)
project_record= Entrez.read(handle)
handle.close()

print(project_record) 

#seleccionamos la informacion especifico 

proj= project_record["DocumentSummarySet"]["DocumentSummary"][0]

#Obtenemos los datos especificos del proyecto 
print(proj.keys()) 

print("\n=== Información del BioProject ===")
print("Tipo de proyecto:", proj["Project_Data_Type"])
print("Metodo del proyecto:", proj["Project_MethodType"])
print("Organismo:", proj["Organism_Name"])
print("Descripción:", proj.get("Project_Description"))


{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['552284'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'PRJNA552284[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'PRJNA552284[All Fields]'}
552284
{'DocumentSummarySet': DictElement({'DocumentSummary': [DictElement({'TaxId': '0', 'Project_Id': '552284', 'Project_Acc': 'PRJNA552284', 'Project_Type': 'Primary submission', 'Project_Data_Type': 'Raw sequence reads', 'Sort_By_ProjectType': '560569', 'Sort_By_DataType': '548649', 'Sort_By_Organism': '882014', 'Project_Subtype': '', 'Project_Target_Scope': 'Multispecies', 'Project_Target_Material': 'Genome', 'Project_Target_Capture': 'Whole', 'Project_MethodType': 'Sequencing', 'Project_Method': '', 'Project_Objectives_List': [{'Project_ObjectivesType': 'Raw Sequence Reads', 'Project_Objectives': ''}], 'Registration_Date': '2019/07/02 00:00', 'Project_Name': '', 'Project_Title': 'Babesia divergens strain:Bd Rouen 1987 | culti

### Consultar con otras bases de datos 

In [21]:
# funcion elink  

handle= Entrez.elink(dbfrom="bioproject", db="gds", id= project_uid)
linksBioProj= Entrez.read(handle)
handle.close()
# este proyecto no cuenta con links por lo que se hara con otro para ejemplificar 
linksBioProj 


proyect_acc="PRJNA16899" 
handle= Entrez.esearch(db="bioproject", term= proyect_acc)
search= Entrez.read(handle)
handle.close()

print(search) 

project_id=search["IdList"] 
print(project_id) 

handle= Entrez.elink(dbfrom="bioproject", db="gds", id= project_id)
linksBioProj= Entrez.read(handle)
handle.close()

linksBioProj

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['16899'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'PRJNA16899[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'PRJNA16899[All Fields]'}
['16899']


[{'LinkSetDbHistory': [], 'LinkSetDb': [], 'ERROR': [], 'DbFrom': 'bioproject', 'IdList': ['16899']}]